In [ ]:
import sys
sys.path.append('..')

from preference_datasets import get_batch_iterator
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import datasets
import matplotlib.pyplot as plt
import random

In [ ]:
lora_dir='PATH TO LORA WEIGHTS HERE'

model = AutoModelForCausalLM.from_pretrained('huggyllama/llama-7b',torch_dtype=torch.float16,device_map='auto')
model = PeftModel.from_pretrained(model, lora_dir)

tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
def maximum_sum_nonadjacent_subsequence(arr):
    N = len(arr)
    dp = [0 for _ in range(N)]
    dp[N - 1] = max(arr[N - 1], 0)
    dp[N - 2] = max(max(arr[N - 1], arr[N - 2]), 0)
    for i in range(N - 3, -1, -1):
        dp[i] = max(max(dp[i + 1], arr[i] + dp[i + 2]), 0)

    result = []
    can_use_next_item_arr=[]
    can_use_next_item = True
    can_use_next_item_arr.append(True)
    for i in range(N - 2):
        if dp[i] == arr[i] + dp[i + 2] and can_use_next_item:
            result.append(1)
            can_use_next_item = False
            can_use_next_item_arr.append(False)
        else:
            result.append(2)
            can_use_next_item = True
            can_use_next_item_arr.append(True)

    if dp[N - 2] == arr[N - 2] and can_use_next_item:
        result.append(1)
        can_use_next_item = False
        can_use_next_item_arr.append(False)
    else:
        result.append(2)
        can_use_next_item = True
        can_use_next_item_arr.append(True)

    if dp[N - 1] == arr[N - 1] and can_use_next_item:
        result.append(1)
    else:
        result.append(2)

    return result,dp,can_use_next_item_arr

def generate_from_prompt(model,prompt,tokenizer,max_length,temperature):
    input_tok=tokenizer(prompt,add_special_tokens=False)
    input_ids=torch.LongTensor(input_tok['input_ids']).cuda()
    attention_mask=torch.LongTensor(input_tok['attention_mask']).cuda()
    tokenized_samples = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length, 
        do_sample=True, 
        temperature=temperature, 
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.batch_decode(tokenized_samples,skip_special_tokens=True)

def generate_random_list(length):
    return [random.randint(-5, 5) for _ in range(length)]


In [ ]:
results_dict={}
batch_size=5
prompt_template="Question: Let's solve input = {}."
for list_len in range(2,15):
    num_right=0
    for _ in range(4):
        arrs=[generate_random_list(list_len) for _ in range(batch_size)]
        true_out=[maximum_sum_nonadjacent_subsequence(arr) for arr in arrs]
        out=generate_from_prompt(model,[prompt_template.format(str(arr)) for arr in arrs],tokenizer,2048,temperature=.01)
        sols=[out_str.split('=')[-1][:-1] for out_str in out]
        num_right+=[sols[i]==str(true_out[i][0]) for i in range(len(sols))].count(True)

    results_dict[list_len]=num_right
print(results_dict)
